In [1]:
print("Starting with Deep Learning")

Starting with Deep Learning


In [2]:
import numpy as np
import pathlib
import struct
import pandas as pd
import gc
import os.path
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from tabulate import tabulate
import csv

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

print("Done")

Done


In [3]:
import numpy as np
import pathlib
import struct
import pandas as pd
import gc
import os.path
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from tabulate import tabulate
import csv


from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

print("Done")

Done


In [4]:
print(tf.__version__)
print(keras.__version__)

2.6.0
2.6.0


In [5]:
print(len(tf.config.list_physical_devices('GPU')))

1


In [6]:
### Usable Files

direction = pathlib.Path("C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Data/Detector97/Filtered/Processed/")

# direction = pathlib.Path("C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Data/Detector97/Unfiltered/Processed/")

file_list = list(direction.iterdir())

file_num = len(file_list)

num_test_files = 2
eighty = round(0.8*file_num)
twenty = file_num-eighty - num_test_files

list_of_file_ids_train = np.arange(eighty, dtype=int)
print(list_of_file_ids_train)
list_of_file_ids_val = np.arange(eighty,eighty+twenty-num_test_files, dtype=int)

list_of_file_ids_test =np.arange(file_num-num_test_files,file_num)



[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]


In [7]:
##### Functions to process the data

        ### Unnormalization of each signal individually
def Unnormalized(batch_signals):
        
        return batch_signals
        
        ### Normalization of each signal individually
def Normalized(batch_signals):

        for i in range(len(batch_signals)):
            batch_signals[i] = batch_signals[i]/np.max(batch_signals[i])
            
        return batch_signals
            
        
        ### Normalization of the entire value by one common denominator      
def Denominator(batch_signals):  
    
        denominator = 3953.48
        batch_signals = batch_signals/denominator
        
        return batch_signals


##### Class

class TrainDataset(tf.data.Dataset):

    def _generator(file_id):  
#         print(f'Using Train Class')
        if(file_id == 0):
#             print("reshuffling")
            np.random.shuffle(list_of_file_ids_train)             

        i_file = list_of_file_ids_train[file_id]

#         print(f'file_id: {file_id}, i_file: {i_file}')
#         print()
        signal_filename = direction/f'{i_file+1}.h5'

        
         # Load the labels and signals from the files
        df = pd.read_hdf(signal_filename,key=None)  
        
        labels1 = df.iloc[:,9].values
        labels2 = df['ToF'].values
        labels = labels1+labels2
        
        signals = df[df.columns[10:-2]].values
        
        
        # Determine how many batches can be made from this file
        num_batches = len(signals) // batch_size

        # Shuffle the signals within the file
        signal_indices = np.arange(len(signals))
        np.random.shuffle(signal_indices)        
        
        # Loop through each batch in the file
        for batch_idx in range(num_batches):
            # Get the signals and labels for this batch
            batch_signal_indices = signal_indices[batch_idx*batch_size:(batch_idx+1)*batch_size]      
 
            batch_signals = signals[batch_signal_indices]
            
            batch_signals = Processing[process](batch_signals)
                
            batch_signals = batch_signals[:,:,np.newaxis] # Can also be done with signals = signals[:,:,np.newaxis]
            batch_labels = labels[batch_signal_indices]

            # Yield the signals and labels as a tuple
            yield batch_signals, batch_labels 
             
    def __new__(cls, file_id):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=(tf.dtypes.float64, tf.dtypes.float64),
            output_shapes=((batch_size, 1998,1), (batch_size, )),
            args=(file_id,)
        )
    
class ValDataset(tf.data.Dataset):

    def _generator(file_id):  
#         print(f'Using Val Class')
        i_file = list_of_file_ids_val[file_id]
    
        signal_filename = direction/f'{i_file+1}.h5'

         # Load the labels and signals from the files
        df = pd.read_hdf(signal_filename,key=None)    
        
        labels1 = df.iloc[:,9].values
        labels2 = df['ToF'].values
        labels = labels1+labels2
        
        signals = df[df.columns[10:-2]].values
        
        
        # Determine how many batches can be made from this file
        num_batches = len(signals) // batch_size

        # Shuffle the signals within the file
        signal_indices = np.arange(len(signals))
        np.random.shuffle(signal_indices)        
        
        # Loop through each batch in the file
        for batch_idx in range(num_batches):
            # Get the signals and labels for this batch
            batch_signal_indices = signal_indices[batch_idx*batch_size:(batch_idx+1)*batch_size]      
 
            batch_signals = signals[batch_signal_indices]
            
            batch_signals = Processing[process](batch_signals)
                
            batch_signals = batch_signals[:,:,np.newaxis] # Can also be done with signals = signals[:,:,np.newaxis]
            batch_labels = labels[batch_signal_indices]

            # Yield the signals and labels as a tuple
            yield batch_signals, batch_labels
             
    def __new__(cls, file_id):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=(tf.dtypes.float64, tf.dtypes.float64),
            output_shapes=((batch_size, 1998,1), (batch_size, )),
            args=(file_id,)
        )

In [8]:
df = pd.read_hdf(file_list[0],key=None)  

labels1 = df.iloc[:,9].values
labels2 = df['ToF'].values
labels = labels1+labels2

print(labels1)

[499.14654541 499.83734131 499.82849121 ... 499.1619873  499.05609131
 499.6555481 ]


In [9]:
# model = keras.models.Sequential()
# model.add(Conv1D(filters=16, kernel_size=5,activation='relu', input_shape=(1998, 1)))
# model.add(Conv1D(filters=8, kernel_size=5,dilation_rate=2,activation='relu'))
# model.add(Conv1D(filters=4, kernel_size=5,dilation_rate=2,activation='relu'))
# model.add(Conv1D(filters=4, kernel_size=5,strides=2,activation='relu'))
# model.add(Conv1D(filters=4, kernel_size=3,strides=2))
# model.add(Flatten())
# model.add(Dense(32,activation='relu'))
# model.add(Dense(16,activation='relu'))
# model.add(Dense(1))

# print(model.summary())


In [10]:
model = keras.models.Sequential()
model.add(Conv1D(filters=16, kernel_size=5,activation='relu', input_shape=(1998, 1)))
model.add(Conv1D(filters=8, kernel_size=5,dilation_rate=2,activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(filters=4, kernel_size=5,dilation_rate=2,activation='relu'))
model.add(Conv1D(filters=4, kernel_size=5,strides=2,activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(filters=4, kernel_size=3,strides=2))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(1))

print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1994, 16)          96        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1986, 8)           648       
_________________________________________________________________
batch_normalization (BatchNo (None, 1986, 8)           32        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1978, 4)           164       
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 987, 4)            84        
_________________________________________________________________
batch_normalization_1 (Batch (None, 987, 4)            16        
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 493, 4)            5

In [11]:
# from tensorflow.keras.regularizers import l1_l2

# # Define the regularization parameters
# l1 = 0.001  # L1 regularization parameter
# l2 = 0.001  # L2 regularization parameter

# # Create the model
# model = Sequential()
# model.add(Conv1D(filters=16, kernel_size=5, activation='relu', input_shape=(1998, 1), kernel_regularizer=l1_l2(l1=l1, l2=l2)))
# model.add(Conv1D(filters=8, kernel_size=5, dilation_rate=2, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
# model.add(BatchNormalization())
# model.add(Conv1D(filters=4, kernel_size=5, dilation_rate=2, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
# model.add(Conv1D(filters=4, kernel_size=5, strides=2, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
# model.add(BatchNormalization())
# model.add(Conv1D(filters=4, kernel_size=3, strides=2, kernel_regularizer=l1_l2(l1=l1, l2=l2)))
# model.add(Flatten())
# model.add(Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
# model.add(Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
# model.add(Dense(1))

# print(model.summary())

In [12]:
# model = keras.models.Sequential()
# model.add(Conv1D(filters=16, kernel_size=5,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=(1998, 1)))
# model.add(Conv1D(filters=8, kernel_size=5,dilation_rate=2,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
# model.add(BatchNormalization())
# model.add(Conv1D(filters=4, kernel_size=5,dilation_rate=2,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
# model.add(Conv1D(filters=4, kernel_size=5,strides=2,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
# model.add(BatchNormalization())
# model.add(Conv1D(filters=4, kernel_size=3,strides=2))
# model.add(Flatten())
# model.add(Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
# model.add(Dense(16,activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
# model.add(Dense(1))

# print(model.summary())



In [13]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, BatchNormalization, Flatten, Dense, MaxPooling1D, Add

# model = Sequential()
# model.add(Conv1D(filters=16, kernel_size=5, activation='relu', input_shape=(1998, 1)))
# model.add(Conv1D(filters=16, kernel_size=5, activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=32, kernel_size=5, dilation_rate=2, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=5, dilation_rate=2, activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=64, kernel_size=5, dilation_rate=2, activation='relu'))
# model.add(Conv1D(filters=64, kernel_size=5, dilation_rate=2, activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=128, kernel_size=5, dilation_rate=2, activation='relu'))
# model.add(Conv1D(filters=128, kernel_size=5, dilation_rate=2, activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(1))

# print(model.summary())

In [14]:
# Pre-processing options
Processing = [Unnormalized,Normalized,Denominator]
process = 2

# Loss Function
loss_function = ['mean_absolute_error','mean_squared_error']
lf = 1

# Training Variables
batch_size = 32
num_epochs = 30

steps_per_epoch = eighty*5000 // batch_size

# Learning Rate
initial_lr = 1e-03
final_lr = 1e-06

# initial_lr = 1e-03
# final_lr = 1e-03

def step_decay(epoch):
    lrate = initial_lr * (final_lr/initial_lr)**(epoch/num_epochs)

    print(f'Current Learning rate: {lrate}')
    return lrate

# Compile the model
model.compile(loss=loss_function[lf], optimizer = keras.optimizers.Adam(initial_lr), metrics=['mean_absolute_error','mean_squared_error'])

# Configuring training dataset
dataset_train = tf.data.Dataset.range(eighty).interleave(
        TrainDataset,
        cycle_length=2,
        num_parallel_calls=2,
        deterministic=True).repeat().prefetch(1)



# Configuring training dataset
dataset_val = tf.data.Dataset.range(twenty-num_test_files).interleave(
        ValDataset,
        cycle_length=2,
        num_parallel_calls=2,
        deterministic=True).prefetch(1)


# Callback Functions
LRS = tf.keras.callbacks.LearningRateScheduler(step_decay)

ES = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True,verbose=1)

CSV = tf.keras.callbacks.CSVLogger("C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Data/Log.csv",
                                separator=",", append=True)

MC_path = pathlib.Path("C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/model_checkpoint.h5")
MC = ModelCheckpoint(
    filepath=MC_path,  # Filepath to save the model weights
    monitor='val_loss',  # Quantity to monitor (e.g., validation loss)
    save_best_only=True,  # Save only the best model based on the monitored quantity
    save_weights_only=True  # Save only the model weights, not the entire model
)

callbacks = [MC,LRS,CSV]




Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead


In [15]:
# history = model.fit(x=dataset_train, validation_data = dataset_val, steps_per_epoch=steps_per_epoch, epochs=num_epochs,callbacks=callbacks)


In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import random

Processing = [Unnormalized,Normalized,Denominator]
process = 1

# Lists to store the results
mae_list = []
mse_list = []
mean_list = []
std_list = []

# Set the number of training runs
num_runs = 2

for run in range(num_runs):
    # Set a new random seed for each run
    random_seed = run  # Change this if you want a different way to generate the seed

    # Set the random seed for NumPy and TensorFlow (or any other random number generators you may be using)
    np.random.seed(random_seed)
    random.seed(random_seed)
    # TensorFlow random seed
    tf.random.set_seed(random_seed)

    # Train the model
    history = model.fit(x=dataset_train, validation_data=dataset_val, steps_per_epoch=steps_per_epoch, epochs=num_epochs, callbacks=callbacks)

    # Load the best model weights
    model.load_weights(MC_path)
    
    for i in list_of_file_ids_test:
        # Load the data and perform necessary preprocessing steps
        df = pd.read_hdf(file_list[i], key=None)
        labels1 = df.iloc[:, 9].values
        labels2 = df['ToF'].values
        labels = labels1 + labels2
        signals = df[df.columns[10:-2]].values
        signals = Processing[process](signals)
        signals = signals[:, :, np.newaxis]

        # Make predictions using the trained model
        predicted = model.predict(signals)
        predicted = np.squeeze(predicted)
        true = labels

        # Calculate MAE, MSE, Mean, and STD
        mae = mean_absolute_error(true, predicted)
        mse = mean_squared_error(true, predicted)
        mean = np.mean(predicted - true)
        std = np.std(predicted - true)

        # Append to the result lists
        mae_list.append(mae)
        mse_list.append(mse)
        mean_list.append(mean)
        std_list.append(std)

# Calculate the average and standard deviation
avg_mae = np.mean(mae_list)
avg_mse = np.mean(mse_list)
avg_mean = np.mean(mean_list)
avg_std = np.mean(std_list)

std_mae = np.std(mae_list)
std_mse = np.std(mse_list)
std_mean = np.std(mean_list)
std_std = np.std(std_list)

print(f"MAE: {mae_list}, {std_mae}")
print(f"MSE: {mse_list}, {std_mse}")
print(f"Mean: {mean_list}, {std_mean}")
print(f"STD: {std_list}, {std_std}")

# Print the results
print(f"Average MAE: {avg_mae:.3f} ± {std_mae:.3f}")
print(f"Average MSE: {avg_mse:.3f} ± {std_mse:.3f}")
print(f"Average Mean: {avg_mean:.3f} ± {std_mean:.3f}")
print(f"Average STD: {avg_std:.3f} ± {std_std:.3f}")

Epoch 1/30
Current Learning rate: 0.001


In [ ]:
model.load_weights(MC_path)

In [ ]:
##### All Analysis Tools:::

# Loss/Val vs Epochs
loss = history.history['loss']
val = history.history['val_loss']

x = np.arange(1,len(loss))

# print(f'Training: {loss} \n')
# print(f'Validation: {val}')

print(f'Train Final: {loss[-1]}')
print(f'Val Final: {val[-1]}')

# Full Training
plt.plot(loss,label='Train')
plt.title(f' Model Performance')
plt.plot(val,label='Validation')
plt.ylabel(f'{loss_function[lf]}')
plt.yscale('log')
plt.xlabel('Epochs')
plt.legend()
plt.grid()
plt.show()

# Full Training
plt.plot(loss,label='Train')
plt.title(f' Model Performance')
plt.plot(val,label='Validation')
plt.ylabel(f'{loss_function[lf]}')
plt.xlabel('Epochs')
plt.legend()
plt.grid()
plt.show()

# Full training without first epoch
plt.plot(x,loss[1:],label='Train')
plt.plot(x,val[1:],label='Validation')
plt.title(f' Model Performance')
plt.ylabel(f'{loss_function[lf]}')
plt.xlabel('Epochs')
plt.legend()
plt.grid()
plt.show()

# Full training without first two epochs
plt.plot(x[1:],loss[2:],label='Train')
plt.plot(x[1:],val[2:],label='Validation')
plt.title(f' Model Performance')
plt.ylabel(f'{loss_function[lf]}')
plt.xlabel('Epochs')
plt.legend()
plt.grid()
plt.show()

# Full training without first three epochs
plt.plot(x[2:],loss[3:],label='Train')
plt.plot(x[2:],val[3:],label='Validation')
plt.title(f' Model Performance')
plt.ylabel(f'{loss_function[lf]}')
plt.xlabel('Epochs')
plt.legend()
plt.grid()
plt.show()

In [ ]:
avg_loss = []


for i in list_of_file_ids_test:
    df = pd.read_hdf(file_list[i],key=None)   

    labels1 = df.iloc[:,9].values
    labels2 = df['ToF'].values
    labels = labels1+labels2

    signals = df[df.columns[10:-2]].values
    signals = Processing[process](signals)
    signals = signals[:,:,np.newaxis]

    # model.predict
    predicted = model.predict(signals)
    predicted = np.squeeze(predicted) # getting rid of extra dimension
    true = labels
    
    test_loss = np.subtract(predicted,true)
    
    avg_test_loss = np.mean(np.abs(test_loss))
    print(avg_test_loss)
    avg_loss.append(avg_test_loss)
    
print(f'Mean Absolute Error: {np.mean(avg_loss)}')

In [ ]:
avg_loss = []

for i in list_of_file_ids_test:
    df = pd.read_hdf(file_list[i], key=None)
    
    labels1 = df.iloc[:, 9].values
    labels2 = df['ToF'].values
    labels = labels1 + labels2
    
    signals = df[df.columns[10:-2]].values
    signals = Processing[process](signals)
    signals = signals[:, :, np.newaxis]
    
    # model.predict
    predicted = model.predict(signals)
    predicted = np.squeeze(predicted)  # getting rid of extra dimension
    true = labels
    
    test_loss = np.subtract(predicted, true)
    
    # Calculate mean squared error (MSE)
    mse = np.mean(np.square(test_loss))
    print("Mean Squared Error:", mse)
    
    avg_loss.append(mse)
    
print(f'Mean Squared Error: {np.mean(avg_loss)}')

In [ ]:
df_test = pd.concat([pd.read_hdf(file_list[i], key=None) for i in list_of_file_ids_test])
unique_mass_test = df_test['A'].unique()

result_dict = {}  # Dictionary to store unique_mass and corresponding avg_test_loss values

# Test Set
for mass_test in unique_mass_test:
    print(f"Unique Mass: {mass_test}")
    df_intermediate = df_test[df_test['A'] == mass_test]
    unique_energy = df_intermediate['Z'].unique()

    for energy in unique_energy:
        print(f"Unique Energy: {energy}")
        
        df_combination = df_intermediate[df_intermediate['Z'] == energy]

        labels1 = df_combination.iloc[:, 9].values
        labels2 = df_combination['ToF'].values
        labels = labels1 + labels2

        signals = df_combination[df_combination.columns[10:-2]].values
        signals = Processing[process](signals)
        signals = signals[:, :, np.newaxis]

        predicted = model.predict(signals)
        predicted = np.squeeze(predicted)
        true = labels

        difference = predicted - true

        # Plotting the distribution
        plt.figure()
        plt.hist(difference, bins=20)
        plt.title(f"A={mass_test}, Z={energy}, Count={len(df_combination)}")
        plt.xlabel("Difference (predicted - true)")
        plt.ylabel("Frequency")

        # Calculate mean and standard deviation
        mean = np.mean(difference)
        std = np.std(difference)

        # Create legend text with mean and standard deviation
        legend_text = f"Mean: {mean:.2f}\nSTD: {std:.2f}"

        # Add legend with the mean and standard deviation
        plt.legend([legend_text])

        plt.show()

        avg_test_loss = np.mean(np.abs(difference))
        count = len(df_combination)

        result_dict[(mass_test, energy)] = {
            'avg_test_loss': avg_test_loss,
            'count': count,
            'mean_test_loss': mean,
            'std_test_loss': std
        }

# Print the result dictionary
for (mass, energy), result in result_dict.items():
    avg_test_loss = result['avg_test_loss']
    count = result['count']
    mean_test_loss = result['mean_test_loss']
    std_test_loss = result['std_test_loss']

    print(f"Unique Mass: {mass}, Unique Energy: {energy}")
    print(f"Avg Test Loss: {avg_test_loss}")
    print(f"Count: {count}")
    print(f"Mean Test Loss: {mean_test_loss}")
    print(f"Standard Deviation Test Loss: {std_test_loss}")
    print()

In [ ]:
# Extract the data for scatter plots
mass_list = []
energy_list = []
mean_loss_list = []
std_loss_list = []

for (mass, energy), result in result_dict.items():
    mass_list.append(mass)
    energy_list.append(energy)
    mean_loss_list.append(result['mean_test_loss'])
    std_loss_list.append(result['std_test_loss'])

    # Scatter plot for Mean Test Loss
plt.figure()
plt.scatter(mass_list, energy_list, c=mean_loss_list, cmap='coolwarm')
plt.colorbar(label='Mean Test Loss')
plt.xlabel('Mass')
plt.ylabel('Energy')
plt.title('Scatter Plot: Mean Test Loss')
plt.show()

# Scatter plot for Standard Deviation
plt.figure()
plt.scatter(mass_list, energy_list, c=std_loss_list, cmap='coolwarm')
plt.colorbar(label='Standard Deviation Test Loss')
plt.xlabel('Mass')
plt.ylabel('Energy')
plt.title('Scatter Plot: Standard Deviation Test Loss')
plt.show()


In [ ]:
import matplotlib.colors as colors
from matplotlib.ticker import LogLocator

# Extract the data for scatter plots
mass_list = []
energy_list = []
mean_loss_list = []
std_loss_list = []

for (mass, energy), result in result_dict.items():
    mass_list.append(mass)
    energy_list.append(energy)
    mean_loss_list.append(result['mean_test_loss'])
    std_loss_list.append(result['std_test_loss'])

# Apply logarithmic transformation to the values
mean_loss_list = [np.log10(val) for val in mean_loss_list]
std_loss_list = [np.log10(val) for val in std_loss_list]

# Create a grid of subplots with 1 row and 2 columns
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Scatter plot for Mean Test Loss
scatter_mean = axs[0].scatter(mass_list, energy_list, c=mean_loss_list, cmap='coolwarm')
plt.colorbar(scatter_mean, ax=axs[0], label='Mean Test Loss (log scale)')
axs[0].set_xlabel('Mass')
axs[0].set_ylabel('Energy')
axs[0].set_title('Scatter Plot: Mean Test Loss')

# Scatter plot for Standard Deviation
scatter_std = axs[1].scatter(mass_list, energy_list, c=std_loss_list, cmap='coolwarm')
plt.colorbar(scatter_std, ax=axs[1], label='Standard Deviation Test Loss (log scale)')
axs[1].set_xlabel('Mass')
axs[1].set_ylabel('Energy')
axs[1].set_title('Scatter Plot: Standard Deviation Test Loss')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.3)

# Show the plot
plt.show()

In [ ]:
import matplotlib.colors as colors

# Extract the data for scatter plots
mass_list = []
energy_list = []
mean_loss_list = []
std_loss_list = []

for (mass, energy), result in result_dict.items():
    mass_list.append(mass)
    energy_list.append(energy)
    mean_loss_list.append(result['mean_test_loss'])
    std_loss_list.append(result['std_test_loss'])

# Create a grid of subplots with 1 row and 2 columns
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Scatter plot for Mean Test Loss
scatter_mean = axs[0].scatter(mass_list, energy_list, c=mean_loss_list, cmap='coolwarm',
                              norm=colors.LogNorm())
plt.colorbar(scatter_mean, ax=axs[0], label='Mean Test Loss')
axs[0].set_xscale('log')
axs[0].set_yscale('log')
axs[0].set_xlabel('Mass')
axs[0].set_ylabel('Energy')
axs[0].set_title('Scatter Plot: Mean Test Loss')

# Scatter plot for Standard Deviation
scatter_std = axs[1].scatter(mass_list, energy_list, c=std_loss_list, cmap='coolwarm',
                             norm=colors.LogNorm())
plt.colorbar(scatter_std, ax=axs[1], label='Standard Deviation Test Loss')
axs[1].set_xscale('log')
axs[1].set_yscale('log')
axs[1].set_xlabel('Mass')
axs[1].set_ylabel('Energy')
axs[1].set_title('Scatter Plot: Standard Deviation Test Loss')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.3)

# Show the plot
plt.show()

In [ ]:
df_test = pd.concat([pd.read_hdf(file_list[i], key=None) for i in list_of_file_ids_test])
unique_mass_test = df_test['A'].unique()

# Test Set
for mass_test in unique_mass_test:
    result_list = []  # List to store results as rows

    df_intermediate = df_test[df_test['A'] == mass_test]
    unique_energy = df_intermediate['Z'].unique()

    for energy in unique_energy:
        df_combination = df_intermediate[df_intermediate['Z'] == energy]

        labels1 = df_combination.iloc[:, 9].values
        labels2 = df_combination['ToF'].values
        labels = labels1 + labels2

        signals = df_combination[df_combination.columns[10:-2]].values
        signals = Processing[process](signals)
        signals = signals[:, :, np.newaxis]

        predicted = model.predict(signals)
        predicted = np.squeeze(predicted)
        true = labels

        difference = predicted - true
        # Calculate mean and standard deviation
        mean = np.mean(difference)
        std = np.std(difference)

        avg_test_loss = np.mean(np.abs(difference))
        count = len(df_combination)

        result_list.append([round(mass_test), round(energy), avg_test_loss, count, mean, std])

    # Create the tabular representation for the current unique mass
    headers = ["Unique Mass", "Unique Energy", "Avg Test Loss", "Count", "Mean Test Loss", "Std Test Loss"]
    tabular_result = tabulate(result_list, headers=headers, floatfmt=".3f", tablefmt="github")

    # Print the tabular result for the current unique mass
    print(f"Tabular representation for Unique Mass: {mass_test}")
    print(tabular_result)
    print()

In [ ]:
# Group the data by unique mass and energy
grouped_data = df_test.groupby(['A'])

# Create an empty list to store the energies for each unique mass
energies_per_mass = []

# Iterate over each unique mass group
for mass, group in grouped_data:
    energies = group['Z'].unique()  # Get the unique energies for the current mass
    energies_per_mass.append((mass, energies))  # Append the unique mass and energies tuple to the list

# Create a list to store the average test losses for each unique mass
avg_test_losses = []

# Create an empty list to store the differences for each unique mass
differences_per_mass = []

# Iterate over the unique masses
for i, (mass, energies) in enumerate(energies_per_mass, 1):
    # Create a list to store the differences for the current mass
    differences = []

    # Iterate over the energies for the current mass
    for energy in energies:
        # Retrieve the data for the current mass and energy
        data = df_test[(df_test['A'] == mass) & (df_test['Z'] == energy)]
        
        # Extract the features (signals) and labels (true values)
        signals = data[data.columns[10:-2]].values
        signals = Processing[process](signals)
        signals = signals[:, :, np.newaxis]
        labels1 = data.iloc[:, 9].values
        labels2 = data['ToF'].values
        labels = labels1 + labels2

        # Predict the labels using the model
        predicted = model.predict(signals)
        predicted = np.squeeze(predicted)

        # Calculate the difference between predicted and true labels
        difference = predicted - labels
        differences.append(difference)  # Store the difference for the current energy

    differences_per_mass.append(differences)  # Store the differences for the current mass

    # Calculate the average test loss for the current mass
    avg_test_loss = np.mean(np.abs(np.concatenate(differences)))  # Concatenate the differences for all energies
    avg_test_losses.append(avg_test_loss)  # Store the average test loss

# Iterate over the unique masses and corresponding differences
for i, (mass, differences) in enumerate(zip(energies_per_mass, differences_per_mass), 1):
    energies = mass[1]  # Get the energies for the current mass

    # Sort the energies in ascending order
    sorted_energies = np.sort(energies)

    # Get the indices to reorder the differences based on the sorted energies
    indices = np.argsort(energies)
    reordered_differences = [differences[index] for index in indices]

    plt.figure(figsize=(150, 6))  # Adjust the figure size as needed
    plt.subplot(1, len(energies_per_mass), i)  # Create a subplot for each unique mass
    plt.boxplot(reordered_differences)
    plt.title(f"Unique Mass: {mass[0]}")
    plt.xlabel("Energy")
    plt.ylabel("Difference (Predicted - True)")

    # Set the x-axis tick labels to the sorted energies
    plt.xticks(range(1, len(sorted_energies) + 1), sorted_energies)

    plt.tight_layout()  # Adjust the spacing between subplots
    plt.show()

# Print the average test losses
for i, (mass, _) in enumerate(energies_per_mass, 1):
    print(f"Unique Mass: {mass}, Average Test Loss: {avg_test_losses[i-1]}")

In [ ]:
# Create a list to store the average test losses for each unique mass
avg_test_losses = []

# Create an empty list to store the differences for each unique mass
differences_per_mass = []

# Iterate over the unique masses
for i, (mass, energies) in enumerate(energies_per_mass, 1):
    # Create a list to store the differences for the current mass
    differences = []

    # Iterate over the energies for the current mass
    for energy in energies:
        # Retrieve the data for the current mass and energy
        data = df_test[(df_test['A'] == mass) & (df_test['Z'] == energy)]
        
        # Extract the features (signals) and labels (true values)
        signals = data[data.columns[10:-2]].values
        signals = Processing[process](signals)
        signals = signals[:, :, np.newaxis]
        labels1 = data.iloc[:, 9].values
        labels2 = data['ToF'].values
        labels = labels1 + labels2

        # Predict the labels using the model
        predicted = model.predict(signals)
        predicted = np.squeeze(predicted)

        # Calculate the difference between predicted and true labels
        difference = predicted - labels
        differences.append(difference)  # Store the difference for the current energy

    differences_per_mass.append(differences)  # Store the differences for the current mass

    # Calculate the average test loss for the current mass
    avg_test_loss = np.mean(np.abs(np.concatenate(differences)))  # Concatenate the differences for all energies
    avg_test_losses.append(avg_test_loss)  # Store the average test loss

# Iterate over the unique masses and corresponding differences
for i, (mass, differences) in enumerate(zip(energies_per_mass, differences_per_mass), 1):
    energies = mass[1]  # Get the energies for the current mass

    # Sort the energies in ascending order
    sorted_energies = np.sort(energies)

    # Get the indices to reorder the differences based on the sorted energies
    indices = np.argsort(energies)
    reordered_differences = [differences[index] for index in indices]

    plt.figure(figsize=(150, 6))  # Adjust the figure size as needed
    plt.subplot(1, len(energies_per_mass), i)  # Create a subplot for each unique mass
    plt.boxplot(reordered_differences)
    plt.title(f"Unique Mass: {mass[0]}")
    plt.xlabel("Energy")
    plt.ylabel("Difference (Predicted - True)")

    # Set the x-axis tick labels to the sorted energies
    plt.xticks(range(1, len(sorted_energies) + 1), sorted_energies)

    # Add a text box with counts for each energy
    counts = [len(d) for d in reordered_differences]
    text_box = "\n".join([f"E{e}: {count}" for e, count in zip(sorted_energies, counts)])
    plt.text(0.95, 0.95, text_box, transform=plt.gca().transAxes, va='top', ha='right',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

    plt.tight_layout()  # Adjust the spacing between subplots
    plt.show()

    


In [ ]:
df_test = pd.concat([pd.read_hdf(file_list[i], key=None) for i in list_of_file_ids_test])
unique_mass_test = df_test['A'].unique()

result_list = []  # List to store results for all unique mass and energy combinations

# Test Set
for mass_test in unique_mass_test:
    df_intermediate = df_test[df_test['A'] == mass_test]
    unique_energy = df_intermediate['Z'].unique()

    for energy in unique_energy:
        df_combination = df_intermediate[df_intermediate['Z'] == energy]

        labels1 = df_combination.iloc[:, 9].values
        labels2 = df_combination['ToF'].values
        labels = labels1 + labels2

        signals = df_combination[df_combination.columns[10:-2]].values
        signals = Processing[process](signals)
        signals = signals[:, :, np.newaxis]

        predicted = model.predict(signals)
        predicted = np.squeeze(predicted)
        true = labels

        difference = predicted - true

        # Calculate mean and standard deviation
        mean = np.mean(difference)
        std = np.std(difference)

        avg_test_loss = np.mean(np.abs(difference))
        count = len(df_combination)

        result_list.append([mass_test, energy, avg_test_loss, count, mean, std])

# Sort the results by avg_test_loss in ascending order
result_list.sort(key=lambda x: x[2])

# Print the 5 best cases
best_cases = result_list[:5]

for case in best_cases:
    mass_test, energy, avg_test_loss, count, mean, std = case

    # Plotting the distribution
    df_combination = df_test[(df_test['A'] == mass_test) & (df_test['Z'] == energy)]
    labels1 = df_combination.iloc[:, 9].values
    labels2 = df_combination['ToF'].values
    labels = labels1 + labels2

    signals = df_combination[df_combination.columns[10:-2]].values
    signals = Processing[process](signals)
    signals = signals[:, :, np.newaxis]

    predicted = model.predict(signals)
    predicted = np.squeeze(predicted)
    true = labels

    difference = predicted - true

    fig, ax = plt.subplots()
    ax.hist(difference, bins=20)
    ax.set_title(f"Best Case - A={mass_test}, Z={energy}, Count={count}")
    ax.set_xlabel("Data")
    ax.set_ylabel("Difference (predicted - true)")
    ax.text(0.75, 0.9, f"Mean: {mean:.2f}\nSTD: {std:.2f}", transform=ax.transAxes, bbox=dict(facecolor='white'))
    plt.show()

    print(f"Unique Mass: {mass_test}, Unique Energy: {energy}")
    print(f"Avg Test Loss: {avg_test_loss}")
    print(f"Count: {count}")
    print(f"Mean Test Loss: {mean}")
    print(f"Standard Deviation Test Loss: {std}")
    print()

In [ ]:
df_test = pd.concat([pd.read_hdf(file_list[i], key=None) for i in list_of_file_ids_test])
unique_mass_test = df_test['A'].unique()

result_list = []  # List to store results for all unique mass and energy combinations

# Test Set
for mass_test in unique_mass_test:
    df_intermediate = df_test[df_test['A'] == mass_test]
    unique_energy = df_intermediate['Z'].unique()

    for energy in unique_energy:
        df_combination = df_intermediate[df_intermediate['Z'] == energy]

        labels1 = df_combination.iloc[:, 9].values
        labels2 = df_combination['ToF'].values
        labels = labels1 + labels2

        signals = df_combination[df_combination.columns[10:-2]].values
        signals = Processing[process](signals)
        signals = signals[:, :, np.newaxis]

        predicted = model.predict(signals)
        predicted = np.squeeze(predicted)
        true = labels

        difference = predicted - true

        # Calculate mean and standard deviation
        mean = np.mean(difference)
        std = np.std(difference)

        avg_test_loss = np.mean(np.abs(difference))
        count = len(df_combination)

        result_list.append([mass_test, energy, avg_test_loss, count, mean, std])

# Sort the results by avg_test_loss in ascending order
result_list.sort(key=lambda x: x[2])

# Print the 5 worst cases
worst_cases = result_list[-5:]

for case in worst_cases:
    mass_test, energy, avg_test_loss, count, mean, std = case

    # Plotting the distribution
    df_combination = df_test[(df_test['A'] == mass_test) & (df_test['Z'] == energy)]
    labels1 = df_combination.iloc[:, 9].values
    labels2 = df_combination['ToF'].values
    labels = labels1 + labels2

    signals = df_combination[df_combination.columns[10:-2]].values
    signals = Processing[process](signals)
    signals = signals[:, :, np.newaxis]

    predicted = model.predict(signals)
    predicted = np.squeeze(predicted)
    true = labels

    difference = predicted - true

    fig, ax = plt.subplots()
    ax.hist(difference, bins=20)
    ax.set_title(f"Worst Case - A={mass_test}, Z={energy}, Count={count}")
    ax.set_xlabel("Data")
    ax.set_ylabel("Difference (predicted - true)")
    ax.text(0.75, 0.9, f"Mean: {mean:.2f}\nSTD: {std:.2f}", transform=ax.transAxes, bbox=dict(facecolor='white'))
    plt.show()

    print(f"Unique Mass: {mass_test}, Unique Energy: {energy}")
    print(f"Avg Test Loss: {avg_test_loss}")
    print(f"Count: {count}")
    print(f"Mean Test Loss: {mean}")
    print(f"Standard Deviation Test Loss: {std}")
    print()

In [ ]:
# Concatenate the test files into a single DataFrame
df_test = pd.concat([pd.read_hdf(file_list[file_id], key=None) for file_id in list_of_file_ids_test])

labels1 = df_test.iloc[:, 9].values
labels2 = df_test['ToF'].values
labels = labels1 + labels2

signals = df_test[df_test.columns[10:-2]].values
signals = Processing[process](signals)
signals = signals[:, :, np.newaxis]

# model.predict
predicted = model.predict(signals)
predicted = np.squeeze(predicted)  # getting rid of extra dimension
true = labels

difference = predicted - true
# Your code to generate the histogram
plt.hist(difference, bins=100)
plt.title("Difference between Predicted and True Values")
plt.xlabel("Difference (Predicted - True) (ns)")
plt.ylabel("Frequency")

# Calculate mean and standard deviation
mean = np.mean(difference)
std = np.std(difference)

# Create legend text with mean and standard deviation
legend_text = f"Mean: {mean:.2f}\nSTD: {std:.2f}"

# Add legend with the mean and standard deviation
plt.legend([legend_text])

plt.show()

In [ ]:
# Concatenate the test files into a single DataFrame
df_test = pd.concat([pd.read_hdf(file_list[file_id], key=None) for file_id in list_of_file_ids_test])

labels1 = df_test.iloc[:, 9].values
labels2 = df_test['ToF'].values
labels = labels1 + labels2

signals = df_test[df_test.columns[10:-2]].values
signals = Processing[process](signals)
signals = signals[:, :, np.newaxis]

# model.predict
predicted = model.predict(signals)
predicted = np.squeeze(predicted)  # getting rid of extra dimension
true = labels

difference = predicted - true
# Plotting the histogram
plt.hist(difference, bins=100)
plt.title("Difference between Predicted and True Values")
plt.xlabel("Difference (Predicted - True) (ns)")
plt.ylabel("Frequency")
plt.yscale('log')  # Set y-axis to logarithmic scale

# Calculate mean and standard deviation
mean = np.mean(difference)
std = np.std(difference)

# Create legend text with mean and standard deviation
legend_text = f"Mean: {mean:.2f}\nSTD: {std:.2f}"

# Add legend with the mean and standard deviation
plt.legend([legend_text])

plt.show()